In [23]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ff_energy.ffe.utils import read_from_pickle
import jax.numpy as jnp
from ff_energy.ffe.plot import plot_energy_MSE, plot_ff_fit
from ff_energy.ffe.structure import atom_key_pairs
from ff_energy.ffe.potential import LJ, akp_indx
from ff_energy.ffe.ff import FF
from ff_energy.ffe.ff_fit import LJ_bound, load_ff, fit_func, fit_repeat
from ff_energy.ffe.data import pairs_data
from ff_energy.ffe.utils import pickle_output, read_from_pickle, str2int




In [24]:
NDE = 10
NLJ = 10
sig_bound = (0.05, 2.5)
ep_bound = (0.001, 0.5)
LJ_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound))
DE_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound), 
            (1, 8), (6, 20))

In [25]:
#  example
# ffpkl = "ECOL_pbe0dz_FOE.ff.pkl"

In [26]:
# Parameters
ffpkl = "mdcm_pbe0dz_clusters.ff.pkl"


#  LJ fit


In [40]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))
_c

FF: LJ water_cluster ELEC harmonic intE (jax_coloumb: True)

In [41]:
# _c.targets

In [42]:
fit_repeat(_c, 
           NLJ, 
           f"{ffpkl}_LJ",
           bounds = LJ_bound,
           loss="jax",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [43]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
1,16.279213,107,242,0,True,Optimization terminated successfully.,"[0.11385642287948355, 1.808702200297021, 0.052...","([[0.11385642287948355, 1.808702200297021, 0.0..."
7,16.285727,151,294,0,True,Optimization terminated successfully.,"[0.05227990133014991, 1.8331754155181081, 0.23...","([[0.05227990133014991, 1.8331754155181081, 0...."
0,16.286459,235,439,0,True,Optimization terminated successfully.,"[0.05000000000000086, 1.818192400427217, 0.130...","([[0.05000000000000086, 1.818192400427217, 0.1..."
9,16.308668,232,452,0,True,Optimization terminated successfully.,"[0.4143515602774177, 1.8038772861071504, 0.006...","([[0.4143515602774177, 1.8038772861071504, 0.0..."
3,16.323795,363,653,0,True,Optimization terminated successfully.,"[0.050002929471845134, 1.8619766754361884, 0.4...","([[0.050002929471845134, 1.8619766754361884, 0..."
8,16.323799,359,619,0,True,Optimization terminated successfully.,"[0.050000270891198705, 1.8618010581565496, 0.5...","([[0.050000270891198705, 1.8618010581565496, 0..."
5,27.281115,333,598,0,True,Optimization terminated successfully.,"[1.664340205832399, 0.9612137272340543, 0.001,...","([[1.664340205832399, 0.9612137272340543, 0.00..."
6,27.281122,204,391,0,True,Optimization terminated successfully.,"[1.664433059771683, 0.9610955443593516, 0.001,...","([[1.664433059771683, 0.9610955443593516, 0.00..."
2,536.786438,59,168,0,True,Optimization terminated successfully.,"[0.1017963771379734, 0.05, 0.00369481758476521...","([[0.1017963771379734, 0.05, 0.003694817584765..."
4,536.786438,53,159,0,True,Optimization terminated successfully.,"[0.05521840853692639, 0.08567559490945773, 0.3...","([[0.05521840853692639, 0.08567559490945773, 0..."


#  DE fit


In [36]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))

In [37]:
fit_repeat(_c, 
           NDE, 
           "test",
           bounds = DE_bound,
           loss="jax_de",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [39]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
1,16.178349,622,1058,0,True,Optimization terminated successfully.,"[0.16172821365751386, 1.8325767858354398, 0.13...","([[0.16172821365751386, 1.8325767858354398, 0...."
5,16.180923,1121,1835,0,True,Optimization terminated successfully.,"[0.4623703400327051, 1.8179016741392386, 0.002...","([[0.4623703400327051, 1.8179016741392386, 0.0..."
0,16.206945,908,1487,0,True,Optimization terminated successfully.,"[0.14620791569498015, 1.7902355800861094, 0.25...","([[0.14620791569498015, 1.7902355800861094, 0...."
7,16.237680,682,1128,0,True,Optimization terminated successfully.,"[0.2235624279668617, 1.7891905068337857, 0.125...","([[0.2235624279668617, 1.7891905068337857, 0.1..."
6,20.811485,1491,2364,0,True,Optimization terminated successfully.,"[2.3233383919551285, 1.0172893203045963, 0.001...","([[2.3233383919551285, 1.0172893203045963, 0.0..."
8,21.054459,789,1307,0,True,Optimization terminated successfully.,"[1.7298452237138855, 1.0969872504655813, 0.001...","([[1.7298452237138855, 1.0969872504655813, 0.0..."
3,21.197947,634,1062,0,True,Optimization terminated successfully.,"[1.267690022287375, 2.4944085049494076, 0.0899...","([[1.267690022287375, 2.4944085049494076, 0.08..."
2,536.786438,21,167,0,True,Optimization terminated successfully.,"[0.10210801510277207, 0.15116488171414028, 0.4...","([[0.10210801510277207, 0.15116488171414028, 0..."
4,536.786438,63,225,0,True,Optimization terminated successfully.,"[0.05088611574405752, 0.25838309128460146, 0.2...","([[0.05088611574405752, 0.25838309128460146, 0..."
9,536.786438,41,207,0,True,Optimization terminated successfully.,"[0.1712342162973508, 0.09172934676293532, 0.13...","([[0.1712342162973508, 0.09172934676293532, 0...."
